In [1]:
import os
import pandas as pd
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from afinn import Afinn
afinn=Afinn(emoticons=True)

In [2]:
df=pd.read_excel('Data_reviews1_to_44_Final.xlsx')
df['Review Date']=pd.to_datetime(df['Review Date'])
df=df.sort_values('Review Date',ascending=False).reset_index()
df=df.drop('index',axis=1)
df.to_csv('forR.csv',sep='|')

In [3]:
files=sorted(os.listdir('/Users/FrankLIUChangxuan/Desktop/Proj/quarter_rev'))

In [11]:
##Overall topics
def getTopicSentiment(file):
    
    d1=pd.read_csv('quarter_rev/'+file,index_col=0)
    vectorizer1=CountVectorizer(stop_words='english')
    X=vectorizer1.fit_transform(d1['Review'])
    terms=vectorizer1.get_feature_names()
    lda=LatentDirichletAllocation(n_components=5,learning_method='online').fit(X)
    topics=[' '.join([terms[i] for i in topic.argsort()[:-5-1:-1]]) for topic_idx,topic in enumerate(lda.components_)]
    mtx=[lda.fit_transform(X[k]) for k in range(X.shape[0])]
    print(topics)

    t0=[mtx[i][0][0] for i in range(len(mtx))]
    t1=[mtx[i][0][1] for i in range(len(mtx))]
    t2=[mtx[i][0][2] for i in range(len(mtx))]
    t3=[mtx[i][0][3] for i in range(len(mtx))]
    t4=[mtx[i][0][4] for i in range(len(mtx))]
    #t5=[mtx[i][0][4] for i in range(len(mtx))]
    #t6=[mtx[i][0][4] for i in range(len(mtx))]
    #t7=[mtx[i][0][4] for i in range(len(mtx))]
    #t8=[mtx[i][0][4] for i in range(len(mtx))]
    #t9=[mtx[i][0][4] for i in range(len(mtx))]
    
    d1=pd.concat([d1,pd.DataFrame({'topic0':t0,'topic1':t1,'topic2':t2,'topic3':t3,'topic4':t4})],axis=1)
    d1=d1.sort_values('Restaurant.name').drop([0])
    
    #sentiment
    r_names=[]
    shr=[]
    time=[]
    score_t0=[]
    score_t1=[]
    score_t2=[]
    score_t3=[]
    score_t4=[]
    #score_t5=[]
    #score_t6=[]
    #score_t7=[]
    #score_t8=[]
    #score_t9=[]
    for r_name in d1['Restaurant.name'].unique():
        r_names.append(r_name)
        time.append(int(file[4:8]+'0'+file[10]))
        r=d1.loc[d1['Restaurant.name']==r_name]
        shr.append(len(r.index)/len(d1.index))
        score_t0.append(afinn.score(''.join([i for i in r.loc[r['topic0']>0.8]['Review']])))
        score_t1.append(afinn.score(''.join([i for i in r.loc[r['topic1']>0.8]['Review']])))
        score_t2.append(afinn.score(''.join([i for i in r.loc[r['topic2']>0.8]['Review']])))
        score_t3.append(afinn.score(''.join([i for i in r.loc[r['topic3']>0.8]['Review']])))
        score_t4.append(afinn.score(''.join([i for i in r.loc[r['topic4']>0.8]['Review']])))
        #score_t5.append(afinn.score(''.join([i for i in r.loc[r['topic5']>0.8]['Review']])))
        #score_t6.append(afinn.score(''.join([i for i in r.loc[r['topic6']>0.8]['Review']])))
        #score_t7.append(afinn.score(''.join([i for i in r.loc[r['topic7']>0.8]['Review']])))
        #score_t8.append(afinn.score(''.join([i for i in r.loc[r['topic8']>0.8]['Review']])))
        #score_t9.append(afinn.score(''.join([i for i in r.loc[r['topic9']>0.8]['Review']])))
    return pd.DataFrame({'mkt_shr':shr,
                         'Restaurant.name':r_names,
                         'Review.date':time,
                         'score_t0':score_t0,'score_t1':score_t1,'score_t2':score_t2,'score_t3':score_t3,'score_t4':score_t4},
                         #'score_t5':score_t5,'score_t6':score_t6,'score_t7':score_t7,'score_t8':score_t8,'score_t9':score_t9})

In [12]:
dfs=[getTopicSentiment(x) for x in files]

['kid changing vendor thoughtful daycare', 'experience food like new didn', 'taqueria 1st healthy ago burritos', 'rabbit attracted jacob conejo unpretentious', 'food good great place mexican', 'exceptionally subjected unacceptable pretentious condescending', 'perfume ruined 06 busser choked', 'paquitos juice sugar blue giant', 'lamb standard birria moose watch', 'taco tacos pork asada carne']
['food good great place service', 'million cammorones caution sooooo fogón', 'delivery funny weird rock honest', 'barrio wallet lost mailed yes', 'enchilada umm chukkis remember half', 'radishes azul cake provide stand', 'service dog password cash sorry', 'tacos place carne asada taco', 'coconut tai mai called pecado', 'nachos steak antiguo rooftop casco']
['incredible passion pm eveerything texmex', 'food place tacos great good', 'fellow vendetta losing rebellion cheerful', 'mazatlán locos papa bistec chille', 'steak beuno pecado pit sounded', 'food good mexican place drinks', 'favor quick bite g

In [13]:
all_names=[df['Restaurant.name'] for df in dfs]
rst_always=set.intersection(*map(set,all_names))
dfs=[df.loc[df['Restaurant.name'].isin(rst_always)] for df in dfs]
final=pd.concat(dfs)
final.to_csv('final.csv',index=False)